# Import packages


In [ ]:
import pandas as pd
import numpy as np
import os
from IPython.display import display
import geopandas as gpd
import glob
import leafmap

# Import and Load Data

In [ ]:
# load the csv files

# specify the path to csv files
path="CSV_Folder"

files=glob.glob(path + "/*.csv")
# defining an empty list to store 
# content
data_frame = pd.DataFrame()
content = []

# checking all the csv files in the 
# specified path
for filename in files:
  
    # reading content of csv file
    # content.append(filename)
    df = pd.read_csv(filename, index_col=None)
    content.append(df)

# converting content to data frame
df_combined = pd.concat(content)

display(df_combined.head())

In [ ]:
#convert the data into a geopandas dataframe
df_aq=gpd.GeoDataFrame(df_combined, geometry=gpd.points_from_xy(df_combined['Site Longitude'],df_combined['Site Latitude']), crs="EPSG:4326")

In [ ]:
display(df_aq.head())

# Create a map to see sensor locations

In [ ]:
# create copy of data
df_aq_sites=df_aq.copy()

In [ ]:
# create summary of sensor site locations
df_aq_sites=df_aq_sites[['Site ID','Local Site Name','CBSA Name', 'County','State', 'State FIPS Code', 'County FIPS Code', 'geometry']].drop_duplicates()

In [ ]:
df_aq_sites

In [ ]:
# create map using leafmap
m=leafmap.Map()
m.add_gdf(df_aq_sites, layer_name="Sensor Sites")
m

In [ ]:
# export air sensors as a geopackage
file_name_sensor_location="aq_sensor_site_locations" #this is the file name
df_aq_sites.to_file(file_name_sensor_location+".gpkg", driver="GPKG")

# Data Cleaning

In [ ]:
# Ensure the index is a datetime index
df_aq['Date_analysis'] = pd.to_datetime(df_aq['Date'])

# Extract year from the date index
df_aq['Year'] = df_aq['Date_analysis'].dt.year

# Extract Month
df_aq['month'] = df_aq['Date_analysis'].dt.month

In [ ]:
month_to_season={
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Fall',
    10: 'Fall',
    11: 'Fall',
    12: 'Winter',
    }
    
df_aq['season'] = df_aq['month'].map(month_to_season) #add seasons based on month
    

In [ ]:
display(df_aq.head())

In [ ]:
#create categorical column based on aqs parameter description

df_aq_messy=df_aq.copy()

df_aq_messy['AQS Parameter Description']=pd.Categorical(df_aq_messy['AQS Parameter Description'], categories=['PM2.5 - Local Conditions','Acceptable PM2.5 AQI & Speciation Mass'], ordered=True)


#sort values based on site ID, date, and aqs parameter code
#drop repeat days based on aqs parameter description

df_aqi_cleaned=df_aq.sort_values(by=['Site ID','Date_analysis','AQS Parameter Description']).drop_duplicates(subset=['Site ID','Date_analysis']).sort_index()

display(df_aqi_cleaned)

In [ ]:
# check to see how many duplicates were removed from the data
print('original dataframe count: ', df_aq['Date'].count())
print('cleaned dataframe count: ', df_aqi_cleaned['Date'].count())

In [ ]:
# changing any negative PM2.5 values into 0
df_aqi_cleaned['Daily Mean PM2.5 Concentration']=df_aqi_cleaned['Daily Mean PM2.5 Concentration'].apply(lambda x: 0 if x<0 else x)

In [ ]:
def aqi_categories(aqi_breakpoints):
    '''
    This function creates the EPA AQI Categories based on the AQI breakpoints. This is based on documentation found here:
    https://document.airnow.gov/technical-assistance-document-for-the-reporting-of-daily-air-quailty.pdf, which was accessed 12/18/2024.

    Inputs:
    -------
    aqi: integer
        The calculated AQI value.

    Returns:
    --------
    string
        The different AQI categories based on the AQI breakpoints are returned.
    
    '''
    if(aqi_breakpoints<=50):
        return "Good"
    elif(51<=aqi_breakpoints<=100):
        return "Moderate"
    elif(101<=aqi_breakpoints<=150):
        return "Unhealthy for Sensitive Individuals"
    elif(151<=aqi_breakpoints<=200):
        return "Unhealthy"
    elif(201<=aqi_breakpoints<=300):
        return "Very Unhealthy"
    elif(301<=aqi_breakpoints):
        return "Hazardous"

In [ ]:
# add air quality index values using the data's Daily AQI Value
df_aqi_cleaned['aqi_category']=df_aqi_cleaned['Daily AQI Value'].apply(aqi_categories) #apply AQI categories

In [ ]:
df_aqi_cleaned.head()

In [ ]:
# convert data into a geopandas dataframe
df_aqi_cleaned=gpd.GeoDataFrame(df_aqi_cleaned, geometry='geometry', crs="EPSG:4326")

In [ ]:
# save the cleaned dataset as a geopackage
file_name_cleaned_data="cleaned_air_quality_data" #this is the file name
df_aqi_cleaned.to_file(file_name_cleaned_data+".gpkg", driver="GPKG")

# Seasonal Analysis

In [ ]:
# calculating seasonal stats

season_stats = (
    df_aqi_cleaned.groupby(['Site ID', 'season', 'Units', 'Local Site Name', 'CBSA Name',
       'State FIPS Code', 'State', 'County FIPS Code', 'County',
       'Site Latitude', 'Site Longitude', 'geometry'])['Daily Mean PM2.5 Concentration']
      .agg(['min', 'max', 'median', 'mean'])
      .reset_index()
)

In [ ]:
season_stats.head()

In [ ]:
def aqi_categories_pm(arithmetic_mean):
    '''
    This function creates the EPA AQI Categories based on the PM 2.5 breakpoints. This is based on documentation found here:
    https://document.airnow.gov/technical-assistance-document-for-the-reporting-of-daily-air-quailty.pdf, which was accessed 12/18/2024.

    Inputs:
    -------
    arithmetic_mean: integer
        The calculated PM 2.5 value.

    Returns:
    --------
    string
        The different AQI categories based on the PM 2.5 breakpoints are returned.
    
    '''
    if(arithmetic_mean<=9.0):
        return "Good"
    elif(9.1<=arithmetic_mean<=35.4):
        return "Moderate"
    elif(35.5<=arithmetic_mean<=55.4):
        return "Unhealthy for Sensitive Individuals"
    elif(55.5<=arithmetic_mean<=125.4):
        return "Unhealthy"
    elif(125.5<=arithmetic_mean<=225.4):
        return "Very Unhealthy"
    elif(225.5<=arithmetic_mean):
        return "Hazardous"


In [ ]:
# calculate the AQI categories based on the min, median, mean, and max PM 2.5 values
season_stats['min_aqi_category']=season_stats['min'].apply(aqi_categories_pm) #apply AQI categories
season_stats['median_aqi_category']=season_stats['median'].apply(aqi_categories_pm) #apply AQI categories
season_stats['mean_aqi_category']=season_stats['mean'].apply(aqi_categories_pm) #apply AQI categories
season_stats['max_aqi_category']=season_stats['max'].apply(aqi_categories_pm) #apply AQI categories

In [ ]:
season_stats.head()

In [ ]:
#convert season stats into geopandas dataframe
season_stats_gdf=gpd.GeoDataFrame(season_stats, geometry='geometry', crs="EPSG:4326")

In [ ]:
season_stats_gdf.head()

In [ ]:
#save data as a geopackage
seasonal_stats_file_name="seasonal_air_quality_analysis"
season_stats_gdf.to_file(seasonal_stats_file_name+".gpkg", driver="GPKG")